In [6]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


In [7]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_group(pappas_only=False)

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [8]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])

In [9]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [18]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

for i in range(10):
    X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, X_STSIMM, label_material, label_visiprog, \
                     test_size=0.65, stratify=label_material)

    res = leave_one_sample_out(X_STSIMM_train, Y_mat_train, X_STSIMM_test, Y_mat_test)
    accuracy_STSIMM.append(res['accuracy'])

    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(np.nan)

    model_vsp_original = LFDA()
    try:
        model_vsp_original.fit(X_train, Y_VSP_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = leave_one_sample_out(X_vsp_original_test, Y_mat_test)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        accuracy_vsp_original.append(np.nan)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(np.nan)

INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
I

In [19]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])
df.loc['average'] = df.mean()
display(df)

,STSIMM,material,visiprog_original,visiprog
0,0.781232,0.985044,0.960997,0.978592
1,0.788856,0.985044,0.967742,0.982405
2,0.783578,0.980645,0.960997,0.977419
3,0.786217,0.987977,0.971261,0.981818
4,0.775953,0.983578,0.954252,0.979472
5,0.774487,0.986804,0.963636,0.982698
6,0.778299,0.987683,0.965982,0.979765
7,0.779472,0.982991,0.961877,0.979765
8,0.784164,0.983871,0.928446,0.971848
9,0.775953,0.982405,0.963636,0.974487


In [ ]:
print(Y_mat_test)